## SQL Refresher with SQLite and Python


### Setup: Creating the In-Memory SQLite Database

In [38]:

import sqlite3
import pandas as pd

# Create in-memory SQLite DB and cursor
conn = sqlite3.connect(":memory:")
cursor = conn.cursor()

# Create tables
query = """
CREATE TABLE customers (
    customer_id INTEGER PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
);

CREATE TABLE orders (
    order_id INTEGER PRIMARY KEY,
    customer_id INTEGER,
    order_date TEXT,
    total_amount REAL,
    FOREIGN KEY(customer_id) REFERENCES customers(customer_id)
);

CREATE TABLE books (
    book_id INTEGER PRIMARY KEY,
    title TEXT,
    genre TEXT,
    qty INTEGER
);

CREATE TABLE users (
    id INTEGER PRIMARY KEY,
    username TEXT
);

CREATE TABLE likes (
    id INTEGER PRIMARY KEY,
    user_id INTEGER,
    post_id INTEGER
);
"""
cursor.executescript(query)

# Insert sample data (expanded for richer results)
cursor.executemany("INSERT INTO customers (first_name, last_name) VALUES (?, ?)", [
    ("Alice", "Smith"),
    ("Bob", "Johnson"),
    ("Charlie", "Lee"),
    ("Diana", "Wang"),
    ("Ethan", "Brown"),
    ("Fiona", "Garcia"),
    ("George", "Martinez"),
    ("Hannah", "Kim"),
    ("Ivan", "Patel"),
    ("Julia", "Nguyen")
])

cursor.executemany("INSERT INTO orders (customer_id, order_date, total_amount) VALUES (?, ?, ?)", [
    (1, "2023-01-15", 150.00),
    (2, "2023-01-17", 200.00),
    (1, "2023-01-20", 50.00),
    (3, "2023-01-22", 300.00),
    (4, "2023-01-25", 120.00),
    (5, "2023-01-27", 80.00),
    (6, "2023-01-29", 220.00),
    (7, "2023-02-01", 175.00),
    (8, "2023-02-03", 90.00),
    (9, "2023-02-05", 60.00),
    (10, "2023-02-07", 400.00),
    (2, "2023-02-10", 130.00),
    (3, "2023-02-12", 210.00),
    (4, "2023-02-14", 95.00),
    (5, "2023-02-16", 180.00)
])

cursor.executemany("INSERT INTO books (title, genre, qty) VALUES (?, ?, ?)", [
    ("Book A", "Fiction", 5),
    ("Book B", "Fiction", 10),
    ("Book C", "Non-Fiction", 8),
    ("Book D", "Sci-Fi", 12),
    ("Book E", "Sci-Fi", 7),
    ("Book F", "Fantasy", 15),
    ("Book G", "Fantasy", 3),
    ("Book H", "Mystery", 9),
    ("Book I", "Mystery", 11),
    ("Book J", "Non-Fiction", 14),
    ("Book K", "Fiction", 6),
    ("Book L", "Fantasy", 8),
    ("Book M", "Sci-Fi", 10),
    ("Book N", "Mystery", 5),
    ("Book O", "Non-Fiction", 13)
])

cursor.executemany("INSERT INTO users (username) VALUES (?)", [
    ("user1",), ("user2",), ("user3",), ("user4",), ("user5",),
    ("user6",), ("user7",), ("user8",), ("user9",), ("user10",)
])

cursor.executemany("INSERT INTO likes (user_id, post_id) VALUES (?, ?)", [
    (1, 1215), (2, 1215), (3, 1216), (1, 1217), (4, 1215), (5, 1216),
    (6, 1217), (7, 1215), (8, 1218), (9, 1218), (10, 1217), (2, 1216),
    (3, 1217), (4, 1218), (5, 1215), (6, 1216), (7, 1217), (8, 1215),
    (9, 1216), (10, 1218), (1, 1218), (2, 1217), (3, 1215), (4, 1216),
    (5, 1217), (6, 1218), (7, 1216), (8, 1217), (9, 1215), (10, 1216)
])

conn.commit()
# No fetch or DataFrame needed after table creation

### Basic SQL SELECT Statement
The `SELECT` statement is used to fetch data from a database. It can retrieve one or more columns from one or more tables.

In [39]:

query = """SELECT * FROM customers;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,customer_id,first_name,last_name
0,1,Alice,Smith
1,2,Bob,Johnson
2,3,Charlie,Lee
3,4,Diana,Wang
4,5,Ethan,Brown
5,6,Fiona,Garcia
6,7,George,Martinez
7,8,Hannah,Kim
8,9,Ivan,Patel
9,10,Julia,Nguyen


### GROUP BY and HAVING
`GROUP BY` aggregates data across rows sharing the same value of specified columns. `HAVING` is used to filter these grouped results.

In [40]:

query = """SELECT genre, SUM(qty) AS total_qty
FROM books
GROUP BY genre
HAVING total_qty > 10;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,genre,total_qty
0,Fantasy,26
1,Fiction,21
2,Mystery,25
3,Non-Fiction,35
4,Sci-Fi,29


### SQL JOINs
JOINs allow us to combine rows from two or more tables based on a related column. The most common type is `INNER JOIN`.

In [41]:

query = """SELECT c.first_name, c.last_name, o.order_id, o.total_amount
FROM customers AS c
JOIN orders AS o ON c.customer_id = o.customer_id;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,first_name,last_name,order_id,total_amount
0,Alice,Smith,1,150.0
1,Bob,Johnson,2,200.0
2,Alice,Smith,3,50.0
3,Charlie,Lee,4,300.0
4,Diana,Wang,5,120.0
5,Ethan,Brown,6,80.0
6,Fiona,Garcia,7,220.0
7,George,Martinez,8,175.0
8,Hannah,Kim,9,90.0
9,Ivan,Patel,10,60.0


### Subquery in WHERE
Subqueries can be used inside a `WHERE` clause to filter results based on a condition returned by another query.

In [42]:

query = """SELECT id, username
FROM users
WHERE id IN (
    SELECT user_id FROM likes WHERE post_id = 1215
);"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,id,username
0,1,user1
1,2,user2
2,3,user3
3,4,user4
4,5,user5
5,7,user7
6,8,user8
7,9,user9


### Subquery in FROM
Subqueries in the `FROM` clause are used to create temporary tables that can be queried further.

In [43]:

query = """SELECT AVG(total_likes)
FROM (
    SELECT post_id, COUNT(id) AS total_likes
    FROM likes
    GROUP BY post_id
) AS like_counts;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,AVG(total_likes)
0,7.5


### Converting SQL Results to Pandas DataFrame
After executing SQL queries, we can convert the result set into a pandas DataFrame for easier manipulation and visualization.

In [44]:

cursor.execute("SELECT * FROM customers")
rows = cursor.fetchall()
columns = [description[0] for description in cursor.description]
df = pd.DataFrame(rows, columns=columns)
df.head()


,customer_id,first_name,last_name
0,1,Alice,Smith
1,2,Bob,Johnson
2,3,Charlie,Lee
3,4,Diana,Wang
4,5,Ethan,Brown


### SQL SELECT Statement (Review)
The `SELECT` statement is fundamental to SQL. It is used to query data from a table. Here are a few variants:

In [45]:

query = """SELECT first_name, last_name FROM customers;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,first_name,last_name
0,Alice,Smith
1,Bob,Johnson
2,Charlie,Lee
3,Diana,Wang
4,Ethan,Brown
5,Fiona,Garcia
6,George,Martinez
7,Hannah,Kim
8,Ivan,Patel
9,Julia,Nguyen


In [46]:

query = """SELECT * FROM orders WHERE total_amount > 100;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,order_id,customer_id,order_date,total_amount
0,1,1,2023-01-15,150.0
1,2,2,2023-01-17,200.0
2,4,3,2023-01-22,300.0
3,5,4,2023-01-25,120.0
4,7,6,2023-01-29,220.0
5,8,7,2023-02-01,175.0
6,11,10,2023-02-07,400.0
7,12,2,2023-02-10,130.0
8,13,3,2023-02-12,210.0
9,15,5,2023-02-16,180.0


In [47]:

query = """SELECT * FROM orders ORDER BY total_amount DESC;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,order_id,customer_id,order_date,total_amount
0,11,10,2023-02-07,400.0
1,4,3,2023-01-22,300.0
2,7,6,2023-01-29,220.0
3,13,3,2023-02-12,210.0
4,2,2,2023-01-17,200.0
5,15,5,2023-02-16,180.0
6,8,7,2023-02-01,175.0
7,1,1,2023-01-15,150.0
8,12,2,2023-02-10,130.0
9,5,4,2023-01-25,120.0


### GROUP BY and HAVING (Review)
`GROUP BY` is used to group rows with the same values in specified columns.
`HAVING` is used to filter groups based on aggregate functions.

In [48]:

query = """SELECT genre, SUM(qty) AS total_qty
FROM books
GROUP BY genre;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,genre,total_qty
0,Fantasy,26
1,Fiction,21
2,Mystery,25
3,Non-Fiction,35
4,Sci-Fi,29


In [49]:

query = """SELECT genre, SUM(qty) AS total_qty
FROM books
GROUP BY genre
HAVING total_qty >= 10;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,genre,total_qty
0,Fantasy,26
1,Fiction,21
2,Mystery,25
3,Non-Fiction,35
4,Sci-Fi,29


### SQL JOINs (Review)
JOINs are used to combine data from two or more tables based on a related column.
- `INNER JOIN`: returns only matching rows
- `LEFT JOIN`: returns all rows from the left table, even if there are no matches in the right table

In [50]:

query = """SELECT c.first_name, o.order_id, o.total_amount
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,first_name,order_id,total_amount
0,Alice,1,150.0
1,Bob,2,200.0
2,Alice,3,50.0
3,Charlie,4,300.0
4,Diana,5,120.0
5,Ethan,6,80.0
6,Fiona,7,220.0
7,George,8,175.0
8,Hannah,9,90.0
9,Ivan,10,60.0


In [51]:

query = """SELECT c.first_name, o.order_id
FROM customers c
LEFT JOIN orders o ON c.customer_id = o.customer_id;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,first_name,order_id
0,Alice,1
1,Alice,3
2,Bob,2
3,Bob,12
4,Charlie,4
5,Charlie,13
6,Diana,5
7,Diana,14
8,Ethan,6
9,Ethan,15


### Subqueries (Review)
Subqueries can be nested inside `SELECT`, `FROM`, `WHERE`, or `HAVING` clauses.
They are used to break down complex queries into manageable parts.

In [52]:

query = """SELECT username FROM users
WHERE id IN (
    SELECT user_id FROM likes WHERE post_id = 1215
);"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,username
0,user1
1,user2
2,user3
3,user4
4,user5
5,user7
6,user8
7,user9


In [53]:

query = """SELECT AVG(total_likes)
FROM (
    SELECT post_id, COUNT(*) AS total_likes
    FROM likes
    GROUP BY post_id
) AS like_counts;"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,AVG(total_likes)
0,7.5


In [54]:

query = """SELECT customer_id, AVG(total_amount) as avg_amt
FROM orders
GROUP BY customer_id
HAVING avg_amt > (
    SELECT AVG(total_amount) FROM orders
);"""
cursor.execute(query)
rows = cursor.fetchall()
columns = [desc[0] for desc in cursor.description]
pd.DataFrame(rows, columns=columns)


,customer_id,avg_amt
0,2,165.0
1,3,255.0
2,6,220.0
3,7,175.0
4,10,400.0
